# Radio Map

In [1]:
import pickle 
import json
import pandas as pd
import numpy as np
import collections

In [2]:
with open('RPs.pkl' , 'rb') as fp:
    d = pickle.load(fp)

In [3]:
with open('meta.json' , 'r') as fp:
    meta = json.load(fp)

In [4]:
'''
d has the following structure -> {location : RSSI value}


location - > room number.location in that room.timestamp.random
RSSI value -> a vector of length len(AP_LIST)
'''

'\nd has the following structure -> {location : RSSI value}\n\n\nlocation - > room number.location in that room.timestamp.random\nRSSI value -> a vector of length len(AP_LIST)\n'

In [5]:
AP_LIST = meta['ap_list']
AP_MAP = {AP_LIST[i] : i for i in range(len(AP_LIST))}
NUM_RECORDS = len(d)
DIRECTIONS =  ['north', 'south', 'east', 'west']

In [6]:
df = pd.DataFrame(AP_LIST, columns = ['AP'])
temp = pd.DataFrame(d.values()).transpose()
temp.columns = d.keys()
df = pd.concat([df, temp] , axis = 1)
df = df.set_index('AP')

In [7]:
df.describe()

,B7.1.north.1674507393.zjg0b,B7.1.north.1674507395.k26zy,B7.1.north.1674507397.pcipr,B7.1.north.1674507398.rs7qu,B7.1.north.1674507400.ouufp,B7.1.east.1674507403.ny1kt,B7.1.east.1674507405.76foq,B7.1.east.1674507407.l2cc6,B7.1.east.1674507408.ta03m,B7.1.east.1674507410.3yp8b,...,B5.4.south.1674507312.2xnka,B5.4.south.1674507314.v4bfk,B5.4.south.1674507315.y4pck,B5.4.south.1674507317.v9v01,B5.4.south.1674507319.9m61s,B5.4.west.1674507323.umd0n,B5.4.west.1674507324.9s3bx,B5.4.west.1674507326.zhzer,B5.4.west.1674507328.i9eco,B5.4.west.1674507329.ob8h2
count,38,59,66,74,85,86,91,95,97,98,...,81,83,86,89,89,89,87,92,96,100
unique,18,25,24,24,26,24,23,25,30,26,...,25,24,21,22,24,24,23,24,22,25
top,-83,-88,-88,-90,-88,-80,-89,-89,-89,-89,...,-87,-85,-92,-76,-90,-84,-90,-84,-89,-89
freq,5,7,5,8,8,8,8,10,10,10,...,7,7,9,9,8,10,11,9,10,10


In [8]:
# dd = pd.DataFrame(AP_LIST, columns = ['AP'])
# for k in list(d.keys()):
#     temp = pd.DataFrame(d[k], columns=[k])
#     dd = pd.concat([dd, temp] , axis = 1)
# dd = dd.set_index('AP')

In [9]:
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [10]:
RP_LIST = sorted(list(set([x[:find_nth(x, '.', 2)] for x in list(df.columns)])))
RP_MAP = {key : i for i, key in enumerate(RP_LIST)}
INV_RP_MAP = {i : key for i, key in enumerate(RP_LIST)}
TIMESTEPS = 5

In [11]:
radio_map = {}

for direction in DIRECTIONS:
    arr = np.ndarray((len(AP_LIST), len(RP_MAP), TIMESTEPS), dtype = np.float16)
    for key in RP_MAP:
        cols = [x for x in df.columns if key + '.' + direction in x]
        temp = df[cols].to_numpy()
        arr[:,RP_MAP[key],:] = temp 

    radio_map[direction] = arr


In [12]:
radio_map['north'].shape

(953, 116, 5)

# Reliability and Similarity

In [13]:
GAMMA = -80  #Threshold above whcih an AP readig is too small
M = 2 #How many of the 5 time readings does the AP value need to be above GAMMA
LAMBDA = 1 / 1000 #Specifies a small amount to be added to hamming distance denominator to avoid zero division error

In [14]:
I = {} #Indicator matrices to check for "reliable" APs

In [15]:
for direction in DIRECTIONS:
    T = (radio_map[direction] > GAMMA).astype(int).sum(axis = 2)
    _I = (T > M).astype(int)
    I[direction] = _I

In [16]:
S = {} #Similarity matrices which stores a score for how "close" two RPs are

In [17]:
def hamming(a, b):
    return 1 / (np.count_nonzero(a!=b) + LAMBDA)

In [18]:
for direction in DIRECTIONS:
    _S = np.ndarray((len(RP_MAP) , len(RP_MAP)))
    _I = I[direction]
    for i in range(len(RP_MAP)):
        for j in range(len(RP_MAP)):
            _S[i][j] = hamming(_I[:, i] , _I[:, j])
    S[direction] = _S

# Stability

In [19]:
delta = {} # Stability matrix for assigning new cluster centroid

In [20]:
psi = {}
for direction in DIRECTIONS:
    _psi = np.sum(radio_map[direction], axis = 2) / TIMESTEPS
    psi[direction] = _psi
    # print(_psi.shape)
    _delta = np.sum((radio_map[direction] - _psi[:, :, np.newaxis]) ** 2 , axis = 2) / (TIMESTEPS-1)
    _delta = np.sum(np.nan_to_num(_delta) * I[direction] , axis = 0) * (1 / (np.sum(I[direction] , axis = 0)))
    # print(_delta)
    delta[direction] = _delta



/var/folders/48/z568rjb95513g5z3q1_tlmbm0000gn/T/ipykernel_22079/142987154.py:7: RuntimeWarning: divide by zero encountered in true_divide
  _delta = np.sum(np.nan_to_num(_delta) * I[direction] , axis = 0) * (1 / (np.sum(I[direction] , axis = 0)))
/var/folders/48/z568rjb95513g5z3q1_tlmbm0000gn/T/ipykernel_22079/142987154.py:7: RuntimeWarning: invalid value encountered in multiply
  _delta = np.sum(np.nan_to_num(_delta) * I[direction] , axis = 0) * (1 / (np.sum(I[direction] , axis = 0)))


# Clustering

In [21]:
ETA = 0.02 #Threshold for similarity for a node to belong to a cluster

In [22]:
from pprint import pprint

In [23]:
CH = collections.defaultdict(lambda : collections.defaultdict(set))
FL = collections.defaultdict(lambda : collections.defaultdict(set))

for direction in DIRECTIONS:
    print(direction , '##################################')
    B = set([i for i in range(len(RP_MAP))])
    Bprime = set([i for i in range(len(RP_MAP))])
    edgeset = set()
    visited = set()
    _S = S[direction]
    k = -1
    while B:
        k += 1
        fl = set()
        node = B.pop()
        
        print('Candidate : ' , INV_RP_MAP[node])
        for j in Bprime:
            if j != node and _S[j,node] >= ETA:
                fl.add(j)
                if j not in visited:
                    visited.add(j)
                else:
                    edgeset.add(j)
        pprint([INV_RP_MAP[x] for x in fl])
        B = B - fl
        CH[direction][k] = set([node])
        FL[direction][k] = fl

    K = k
    for k in range(K+1):
        temp = (CH[direction][k] | FL[direction][k])
        cluster = list(temp - edgeset)
        stabilities = delta[direction][cluster]
        CH[direction][k] = set([cluster[np.argmin(stabilities)]])
        FL[direction][k] = temp - CH[direction][k]



north ##################################
Candidate :  A10.1
['A9.2', 'A10.3', 'A10.4', 'A9.3', 'A9.4']
Candidate :  A10.2
['A9.2', 'A10.3', 'A10.4', 'A9.3', 'A9.4']
Candidate :  A2.1
['A3.1', 'A2.2', 'A2.3', 'A2.4']
Candidate :  A3.2
['A2.2',
 'A2.3',
 'A2.4',
 'A3.1',
 'A3.3',
 'A3.4',
 'A4.1',
 'A4.2',
 'A4.3',
 'A4.4',
 'A5.1',
 'A5.2',
 'A5.4',
 'A6.2']
Candidate :  A5.3
['A3.3', 'A4.4', 'A5.1', 'A5.2', 'A5.4', 'A6.1', 'A6.2', 'A6.3', 'A6.4', 'A7.1']
Candidate :  A7.2
['A2.4',
 'A5.2',
 'A6.1',
 'A9.1',
 'B1.1',
 'B1.2',
 'B1.3',
 'B1.4',
 'B1.5',
 'B1.6',
 'B2.1',
 'B2.2',
 'B2.3',
 'B2.4',
 'B2.5',
 'B2.6',
 'B3.1',
 'B3.2',
 'B3.3',
 'B3.4',
 'B4.1',
 'B4.2',
 'B4.3',
 'B4.4',
 'B5.1',
 'B5.2',
 'B5.3',
 'B5.4',
 'B7.1',
 'B7.2',
 'B7.3',
 'B7.4',
 'B8.1',
 'B8.2',
 'B8.3',
 'B8.4',
 'C2.1',
 'C2.2',
 'C2.3',
 'C2.5',
 'C2.8',
 'C3.10',
 'C3.11',
 'C3.3',
 'C3.4',
 'C3.5',
 'C3.7',
 'C3.9',
 'C5.1',
 'C5.2',
 'C5.3',
 'C7.1',
 'C7.2',
 'C7.3',
 'C7.4',
 'C7.5',
 'C7.6',
 'C7.7',

In [24]:
clusters = {}
for temp_dir in DIRECTIONS:
    temp = {}
    for key in range(len(CH[temp_dir])):
        temp[list(CH[temp_dir][key])[0]] = FL[temp_dir][key] 
    clusters[temp_dir] = temp

In [2]:
import pickle as pkl

In [26]:
with open('/Users/mukund/Desktop/clusters.pkl' , 'wb') as fp: 
    pkl.dump(clusters, fp)

In [27]:
with open('/Users/mukund/Desktop/rm.pkl' , 'wb') as fp:
    pkl.dump(psi, fp)

In [28]:
with open('/Users/mukund/Desktop/mapping.pkl' , 'wb') as fp: 
    pkl.dump(RP_MAP, fp)

In [3]:
with open('/Users/mukund/Desktop/clusters.pkl' , 'rb') as fp: 
    c = pkl.load(fp)

In [4]:
c

{'north': {0: {2, 3, 33, 34, 35},
  1: {2, 3, 33, 34, 35},
  4: {5, 6, 7, 8},
  13: {5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 21},
  23: {10, 15, 16, 17, 18, 19, 20, 21, 22, 24},
  25: {7,
   17,
   20,
   32,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   71,
   72,
   74,
   77,
   80,
   81,
   83,
   84,
   85,
   87,
   89,
   90,
   100,
   101,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115},
  31: {26, 27, 28, 29, 30},
  69: {70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 82, 88},
  99: {91, 92, 93, 94, 95, 96, 97, 98, 101, 102, 103, 104, 105, 106, 107},
  86: {79, 80, 81, 82, 83, 84, 85, 87, 88, 89}},
 'south': {1: {0, 2, 3, 32, 33, 34, 35},
  5: {4, 6, 7},
  10: {6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22},
  17: {16, 18, 19, 20, 21, 22, 23},
  28: {18, 20, 21, 22, 